# Libraries

In [1]:
from IPython.display import clear_output
!pip install evaluate
clear_output()

In [2]:
# Core
import pandas as pd
import numpy as np

# Split data
from sklearn.model_selection import train_test_split

# Fine tuning
from datasets import Dataset, load_metric
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer 
import evaluate
from transformers import pipeline

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


# Data Preparation

**1. Data Collection**

In [3]:
# Read csv file
train = pd.read_csv('/kaggle/input/dac-itfest-processed/train-processed.csv')
test = pd.read_csv('/kaggle/input/dac-itfest-processed/test-processed.csv')

In [4]:
# Konversi label ke int
conversion_dict = {
    'sadness':0,
    'anger':1,
    'love':2,
    'fear':3,
    'joy':4
}

train['label'] = train['label'].map(conversion_dict)
test['label'] = test['label'].map(conversion_dict)

**2. Feature Engineering**

**3. Split Data**

In [5]:
# Split train dataset
train, valid = train_test_split(train, test_size=0.2, random_state=42)

# Modeling

**1. Prepare Dataset**

In [6]:
# Convert to dataset object
train_dataset = Dataset.from_pandas(train)
validation_dataset = Dataset.from_pandas(valid)

# Delete index from feature
train_dataset = train_dataset.remove_columns("__index_level_0__")
validation_dataset = validation_dataset.remove_columns("__index_level_0__")

In [7]:
# define model tokenizer
tokenizer = AutoTokenizer.from_pretrained("indolem/indobertweet-base-uncased")

# function to tokenize input
def tokenize_function(examples):
    return tokenizer(examples["tweet"], padding="max_length", truncation=True, max_length = 512)


tokenized_train = train_dataset.map(tokenize_function, batched=True)
tokenized_valid = validation_dataset.map(tokenize_function,batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

**2. Train Preparation**

In [8]:
# Define model
model = AutoModelForSequenceClassification.from_pretrained("indolem/indobertweet-base-uncased", num_labels=5)

# Training arguments
training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

# Metric
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at indolem/indobertweet-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**3. Fine-tuning Model**

In [9]:
# Define trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_valid,
    compute_metrics=compute_metrics,
)

In [10]:
# training process
trainer.train()

UsageError: api_key not configured (no-tty). call wandb.login(key=[your_api_key])

In [ ]:
# Save model's weight
trainer.save_model("indobertweet-fine-tuned")

**4. Predict**

In [ ]:
# Initialize fine-tuned model
new_model = AutoModelForSequenceClassification.from_pretrained("/kaggle/working/indobertweet-fine-tuned")
tokenizer = AutoTokenizer.from_pretrained("indolem/indobertweet-base-uncased")

# Inisialisasi pipeline dengan tokenizer yang sudah diinisialisasi
clf = pipeline("text-classification", model=new_model, tokenizer=tokenizer)

In [ ]:
# Function to predict label in test
def predict_text(row):
    text = row['tweet']  
    prediction = clf(text)
    return prediction

In [ ]:
# predicting process
test['label'] = test.apply(predict_text,axis=1)

In [ ]:
# Get only label
test['label'] = test['label'].apply(lambda x: x[0]['label'])

# Konvert label ke bentuk awal
label_conversion = {
    'LABEL_0': 'sadness',
    'LABEL_1': 'anger',
    'LABEL_2': 'love',
    'LABEL_3': 'fear',
    'LABEL_4': 'joy'
}
test['label'] = test['label'].map(label_conversion)

# Save Predicted Value

In [ ]:
test.to_csv("test-predicted-slang-indobertweet.csv",index=False)

NOTES :
- Model : INDOBERT, LLM (IndoBART,IndoGPT), IndoBertEmotioClassification, Indo RoBERTa, Indo ALBERT, Word2Vec + ML, GPT-2 Indo, **IndoBERTweet**
- Addtional Dataset : IndoNLU
- Analisis hasi berdasarkan topik
- Analisis hasil prediksi berdasarkan kata yang sering muncul tiap label
- Analisi akurasi tiap metode ( Pakai konversi bahasa tidak baku vs tidak, ketika pakai model yang berbeda, ketika pakai spell correction vs tidak )